In [1]:
!pip install pyclesperanto napari-simpleitk-image-processing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/

In [2]:
import numpy as np
import os
import glob
import tifffile
from skimage.exposure import rescale_intensity
from google.colab import drive
import pyclesperanto as cle  # using pyclesperanto instead of pyclesperanto_prototype
import napari_simpleitk_image_processing as nsitk

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/holes_1.4Pa-A-2/WNLA/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all files in the input directory
input_files = glob.glob(os.path.join(input_dir, "*.tif"))
input_files.extend(glob.glob(os.path.join(input_dir, "*.tiff")))

print(f"Found {len(input_files)} .tif/.tiff files in {input_dir}")
if len(input_files) == 0:
    print("No .tif/.tiff files found. Please check the input directory.")
    # Option to look for other file types
    all_files = glob.glob(os.path.join(input_dir, "*"))
    if len(all_files) > 0:
        print(f"Found {len(all_files)} files of other types in the directory.")
        print("Available files:")
        for file in all_files:
            print(f" - {os.path.basename(file)}")

# Process each file
for input_path in input_files:
    filename = os.path.basename(input_path)
    base_filename = os.path.splitext(filename)[0]

    # Define output filename using the requested naming convention
    output_filename = f"{base_filename}_hole_projection.tif"
    output_path = os.path.join(output_dir, output_filename)

    print(f"\nProcessing: {filename}")
    print(f"Output will be saved as: {output_filename}")

    # Load the image
    try:
        img = tifffile.imread(input_path)
        print(f"Image loaded successfully, shape: {img.shape}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Handle 4D images (C, Z, Y, X)
    if len(img.shape) == 4:
        print(f"Detected 4D image with shape {img.shape}")
        print("Expected format: (Channels, Z-slices, Height, Width)")

        # Extract first channel for processing
        print("Extracting first channel (index 0)...")
        img_channel = img[0]  # Shape: (Z, Y, X)
        print(f"Channel extracted, shape: {img_channel.shape}")

        # Step 1: Adaptive histogram equalization using napari_simpleitk_image_processing
        print("1. Applying adaptive histogram equalization...")
        try:
            # Using nsitk for adaptive histogram equalization (same parameters as in your example)
            img_equalized = nsitk.adaptive_histogram_equalization(img_channel, 1.0, 1.0, 0, 0, 0)
            print("  napari_simpleitk adaptive histogram equalization succeeded")
        except Exception as e:
            print(f"  napari_simpleitk failed: {e}")
            # Fallback to SimpleITK if available, or skip this step
            print("  Skipping adaptive histogram equalization due to error")
            img_equalized = img_channel  # Use original if adaptive histogram fails

        # Step 2: Gamma correction using pyclesperanto
        print("2. Applying gamma correction...")
        try:
            # Using pyclesperanto for gamma correction (same parameters as in your example)
            img_gamma = cle.gamma_correction(img_equalized, None, 1.0)
            print("  pyclesperanto gamma correction succeeded")
        except Exception as e:
            print(f"  pyclesperanto gamma correction failed: {e}")
            # Fallback to a simple power law transformation
            img_gamma = img_equalized
            print("  Skipping gamma correction due to error")

        # Step 3: Sum z-projection using pyclesperanto
        print("3. Applying sum z-projection...")
        try:
            # Using pyclesperanto for z-projection (same as in your example)
            img_projected = cle.sum_z_projection(img_gamma)
            print("  pyclesperanto sum_z_projection succeeded")
        except Exception as e:
            print(f"  pyclesperanto z-projection failed: {e}")
            # Fallback to numpy
            img_projected = np.sum(img_gamma, axis=0)
            print("  Used numpy fallback for z-projection")

        # Step 4: Final enhancements on the projected image
        print("4. Applying final enhancements...")
        # Normalize to 0-1 range for better visibility
        img_final = rescale_intensity(img_projected, out_range=(0, 1))

        # Save the final result
        tifffile.imwrite(output_path, img_final.astype(np.float32))
        print(f"  Saved final result as {output_filename}")

    else:
        # Handle 2D or 3D images
        # If 3D image, check if it's (Z, Y, X) or (Y, X, C)
        if len(img.shape) == 3:
            if img.shape[2] <= 4:  # Likely (Y, X, C) format with RGB/RGBA
                print(f"Detected color image with shape {img.shape}")
                # Take first channel
                img_channel = img[:, :, 0]
                print(f"Extracted first color channel, shape: {img_channel.shape}")
            else:  # Likely (Z, Y, X) format
                print(f"Detected 3D image with shape {img.shape}")
                # Keep as is for 3D processing
                img_channel = img
        else:
            # 2D image
            print(f"Detected 2D image with shape {img.shape}")
            img_channel = img

        # Processing steps depend on dimensionality
        if len(img_channel.shape) == 3:  # 3D image (Z, Y, X)
            # Apply the napari and pyclesperanto functions
            print("1. Applying adaptive histogram equalization...")
            try:
                # Using nsitk for adaptive histogram equalization
                img_equalized = nsitk.adaptive_histogram_equalization(img_channel, 1.0, 1.0, 0, 0, 0)
                print("  napari_simpleitk adaptive histogram equalization succeeded")
            except Exception as e:
                print(f"  napari_simpleitk failed: {e}")
                # Fallback to original image
                img_equalized = img_channel
                print("  Skipping adaptive histogram equalization due to error")

            # Gamma correction
            print("2. Applying gamma correction...")
            try:
                # Using pyclesperanto
                img_gamma = cle.gamma_correction(img_equalized, None, 1.0)
                print("  pyclesperanto gamma correction succeeded")
            except Exception as e:
                print(f"  pyclesperanto approach failed: {e}")
                # Fallback to original
                img_gamma = img_equalized
                print("  Skipping gamma correction due to error")

            # Z-projection
            print("3. Applying sum z-projection...")
            try:
                # Using pyclesperanto
                img_projected = cle.sum_z_projection(img_gamma)
                print("  pyclesperanto sum_z_projection succeeded")
            except Exception as e:
                print(f"  pyclesperanto z-projection failed: {e}")
                # Fallback to numpy
                img_projected = np.sum(img_gamma, axis=0)
                print("  Used numpy fallback for z-projection")

            # Final enhancements
            print("4. Applying final enhancements...")
            img_final = rescale_intensity(img_projected, out_range=(0, 1))

            # Save the final result
            tifffile.imwrite(output_path, img_final.astype(np.float32))
            print(f"  Saved final result as {output_filename}")

        else:  # 2D image
            # Apply operations directly
            print("1. Applying adaptive histogram equalization...")
            try:
                # Using nsitk for adaptive histogram equalization
                img_equalized = nsitk.adaptive_histogram_equalization(img_channel, 1.0, 1.0, 0, 0, 0)
                print("  napari_simpleitk adaptive histogram equalization succeeded")
            except Exception as e:
                print(f"  napari_simpleitk failed: {e}")
                # Fallback to original image
                img_equalized = img_channel
                print("  Skipping adaptive histogram equalization due to error")

            # Gamma correction
            print("2. Applying gamma correction...")
            try:
                # Using pyclesperanto
                img_gamma = cle.gamma_correction(img_equalized, None, 1.0)
                print("  pyclesperanto gamma correction succeeded")
            except Exception as e:
                print(f"  pyclesperanto approach failed: {e}")
                # Fallback to original
                img_gamma = img_equalized
                print("  Skipping gamma correction due to error")

            # This is the final result for 2D images
            print("3. Applying final enhancements...")
            img_final = rescale_intensity(img_gamma, out_range=(0, 1))

            # Save the final result
            tifffile.imwrite(output_path, img_final.astype(np.float32))
            print(f"  Saved final result as {output_filename}")

    print(f"Completed processing: {filename}")

print("\nAll processing completed!")

/usr/local/lib/python3.11/dist-packages/pyclesperanto/_core.py:105: RuntimeWarning: No backend available. Please install either OpenCL or CUDA on your system.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pyclesperanto/_core.py:172: RuntimeWarning: Error while initialising pyclesperanto: No backend available.

No GPU Backend found.

pyclesperanto requires either CUDA or OpenCL libraries to be installed on your system to work.
Please ensure you have the appropriate drivers installed and up-to-date.

Alternatively, you may need to install the following additional package:
- MacOS: `conda install -c conda-forge ocl_icd_wrapper_apple`
- Linux: `conda install -c conda-forge ocl-icd-system`
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/napari_tools_menu/__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class a

Mounted at /content/drive
Found 1 .tif/.tiff files in /content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial

Processing: denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018 (2).tif
Output will be saved as: denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018 (2)_hole_projection.tif
Image loaded successfully, shape: (3, 13, 1024, 1024)
Detected 4D image with shape (3, 13, 1024, 1024)
Expected format: (Channels, Z-slices, Height, Width)
Extracting first channel (index 0)...
Channel extracted, shape: (13, 1024, 1024)
1. Applying adaptive histogram equalization...
  napari_simpleitk adaptive histogram equalization succeeded
2. Applying gamma correction...
  pyclesperanto gamma correction failed: Backend not selected.
  Skipping gamma correction due to error
3. Applying sum z-projection...
  pyclesperanto z-projection failed: Backend not selected.
  Used numpy fallback for z-projection
4. Applying final enhancements...
  Saved final result as denoised_1.4Pa_A1_20dec21_20xA_L2RA_

In [ ]:
import numpy as np
import os
import glob
import tifffile
from scipy import ndimage
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/holes_1.4Pa-A-2/NLA/'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/holes_1.4Pa-A-2/LA/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all processed files in the input directory
processed_files = glob.glob(os.path.join(input_dir, "*_hole_projection.tif"))

print(f"Found {len(processed_files)} processed files in {input_dir}")
if len(processed_files) == 0:
    print("No processed files found. Please check the input directory.")
    # Option to look for other file types
    all_files = glob.glob(os.path.join(input_dir, "*"))
    if len(all_files) > 0:
        print(f"Found {len(all_files)} files of other types in the directory.")
        print("Available files:")
        for file in all_files:
            print(f" - {os.path.basename(file)}")

# Process each file
for file_path in processed_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    # Save the illumination corrected image to the output directory
    output_path = os.path.join(output_dir, filename)
    tifffile.imwrite(output_path, illumination_corrected.astype(np.float32))
    print(f"  Saved illumination corrected image to: {output_path}")

print("\nAll illumination correction completed!")

Mounted at /content/drive
Found 18 processed files in /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/holes_1.4Pa-A-2/NLA/

Processing: denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq005_hole_projection.tif
Image loaded successfully, shape: (1024, 1024)
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
  Saved illumination corrected image to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/holes_1.4Pa-A-2/LA/denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq005_hole_projection.tif

Processing: denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq004_hole_projection.tif
Image loaded successfully, shape: (1024, 1024)
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
  Saved illumination corrected image to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/holes_1.4Pa-A-2/LA/denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq004_hole_projection.tif

Processing: 